In [ ]:
import numpy as np
from scipy.special import comb
from itertools import permutations

from envs.vases_grid import VasesGrid, VasesEnvState, print_state, str_to_state, state_to_str
from envs.utils import unique_perm, zeros_with_ones
from envs.vases_spec import VasesEnvState2x3, VasesEnvSpec2x3, VasesEnvState3x3, VasesEnvSpec3x3, VasesEnvState2x3Broken, VasesEnvSpec2x3Broken

from value_iter_and_policy import vi_boltzmann, vi_boltzmann_deterministic

In [ ]:
def compute_D_last_step_discrete(mdp, policy, p_0, T, verbose=False):
    '''
    computes the last-step occupancy measure 
    '''
    D_prev = p_0 
    
    t = 0
    for t in range(T):
        
        # for T-step OM we'd do D=np.copy(P_0). However, we want the last step one, so:
        D = np.zeros_like(p_0)
        
        for s in range(mdp.nS):
            for a in range(mdp.nA):
                # due to env being deterministic, sprime=self.P[s][a][1] and p_sprime=1
                D[mdp.P[s][a][1]] += D_prev[s] * policy[s, a] 
                    
        D_prev = np.copy(D)
        if verbose is True: print(D)
    return D

In [ ]:
def OM_method(mdp, current_state, p_0, horizon, temperature=1, epochs=1, learning_rate=0.2, r_vec=None):
    '''
    Modified MaxCausalEnt that maximizes last step occupancy measure for the current state
    '''
    
    if r_vec is None:
        r_vec = .01*np.random.randn(mdp.feature_matrix.shape[1])
        print('Initial reward vector: {}'.format(r_vec))
        
    for i in range(epochs):
        
            # Compute the Boltzmann rational policy \pi_{s,a} = \exp(Q_{s,a} - V_s) 
            V, Q, policy = vi_boltzmann_deterministic(mdp, 1, mdp.feature_matrix @ r_vec, horizon, temperature) 
            
            D = compute_D_last_step_discrete(mdp, policy, p_0, horizon)   
            dL_dr_vec = -(current_state - D) @ mdp.feature_matrix

            # Gradient descent; gradiend may not be the actual gradient -- have to check the math,
            # bit this should do the matching correctly
            r_vec = r_vec - learning_rate * dL_dr_vec
            
            if i%20==0:
                print('Epoch {}'.format(i))
                print('Reward vector: {}'.format(r_vec))
                # print('Policy: {}'.format(policy))
                # print('Last-step D: {} \n'.format(D))

    return r_vec

In [ ]:
def experiment_grid(env,
                 horizon=22, #number of timesteps we assume the expert has been acting previously
                 temperature_irl=1,
                 learning_rate=.1,
                 epochs = 500):

    print('Number of states: ', env.deterministic_T.shape[0])
    print('Initial state:')
    print_state(env.init_state)

    p_0=np.zeros(env.nS)
    p_0[env.state_num[state_to_str(env.init_state)]] = 1
    
    current_state = np.copy(p_0)
    
    r_vec = OM_method(env, current_state, p_0, horizon, temperature_irl, epochs, learning_rate)
    print('Final reward weights: ', r_vec)
    return r_vec

In [ ]:
np.random.seed(1)

env2x3 = VasesGrid(VasesEnvSpec2x3(), VasesEnvState2x3())
experiment_grid(env2x3)

In [ ]:
np.random.seed(1)

env2x3_broken = VasesGrid(VasesEnvSpec2x3Broken(), VasesEnvState2x3Broken())
experiment_grid(env2x3_broken)